In [1]:
import sqlite3
import requests
import sys
sys.path.append("..")
from rcn_py import rsd
from rcn_py import orcid
from rcn_py import openalex

In [3]:
# Create project table, project member table, software table, software contributor table.
rsd.create_rsd_database("/Users/jennifer/RCSNA/data/rsd.db")

'Created'

In [5]:
# Data storage
rsd.store_rsd("/Users/jennifer/RCSNA/data/rsd.db")

'Done'

In [169]:
# a test for reading the database
con = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
cur = con.cursor()

In [170]:
res = cur.execute("SELECT id, title, description FROM project")
project_description = res.fetchall()

In [171]:
cur.close()

In [10]:
proj_clusters, proj_idx2topics = rsd.project_software_cluster("/Users/jennifer/RCSNA/data/rsd.db", 'project')
soft_clusters, soft_idx2topics = rsd.project_software_cluster("/Users/jennifer/RCSNA/data/rsd.db", 'software')

In [12]:
au_group_proj = rsd.author_cluster("/Users/jennifer/RCSNA/data/rsd.db", "project", proj_clusters)
au_group_soft = rsd.author_cluster("/Users/jennifer/RCSNA/data/rsd.db", "software", soft_clusters)

In [13]:
au_group_proj

{'0000-0001-5107-3531': 0,
 '0000-0001-5196-5791': 1,
 '0000-0001-5208-4283': 0,
 '0000-0001-5237-9583': 1,
 '0000-0001-5239-5321': 0,
 '0000-0001-5240-9770': 0,
 '0000-0001-5267-3002': 1,
 '0000-0001-5283-1330': 0,
 '0000-0001-5301-3276': 0,
 '0000-0001-5301-4981': 0,
 '0000-0001-5338-5299': 0,
 '0000-0001-5371-5876': 0,
 '0000-0001-5387-341X': 0,
 '0000-0001-5397-1503': 0,
 '0000-0001-5450-4333': 0,
 '0000-0001-5468-1030': 0,
 '0000-0001-5468-2025': 0,
 '0000-0001-5514-6021': 1,
 '0000-0001-5540-2822': 0,
 '0000-0001-5565-7577': 0,
 '0000-0001-5588-7903': 0,
 '0000-0001-5619-4017': 0,
 '0000-0001-5662-2843': 0,
 '0000-0001-5666-1658': 0,
 '0000-0001-5676-8834': 0,
 '0000-0001-5680-5957': 0,
 '0000-0001-5817-7675': 0,
 '0000-0001-5839-0302': 0,
 '0000-0001-5852-4155': 0,
 '0000-0001-5974-6219': 1,
 '0000-0001-6005-3110': 0,
 '0000-0001-6076-4406': 0,
 '0000-0001-6090-0535': 0,
 '0000-0001-6128-6979': 1,
 '0000-0001-6159-041X': 1,
 '0000-0001-6350-2380': 2,
 '0000-0001-6366-2173': 0,
 

In [15]:
link_proj = rsd.rsd_relationship("/Users/jennifer/RCSNA/data/rsd.db", "project")
link_soft = rsd.rsd_relationship("/Users/jennifer/RCSNA/data/rsd.db", "software")

In [16]:
link_proj

[('0000-0002-0948-1176', '0000-0002-8276-8819'),
 ('0000-0002-0948-1176', '0000-0002-4411-6932'),
 ('0000-0002-8276-8819', '0000-0002-4411-6932'),
 ('0000-0002-0246-8534', '0000-0003-0087-6021'),
 ('0000-0002-0246-8534', '0000-0001-9795-7981'),
 ('0000-0002-0246-8534', '0000-0002-5821-2060'),
 ('0000-0003-0087-6021', '0000-0001-9795-7981'),
 ('0000-0003-0087-6021', '0000-0002-5821-2060'),
 ('0000-0001-9795-7981', '0000-0002-5821-2060'),
 ('0000-0002-6371-2991', '0000-0002-1211-4238'),
 ('0000-0002-6371-2991', '0000-0002-2888-4202'),
 ('0000-0002-6371-2991', '0000-0002-1030-0512'),
 ('0000-0002-6371-2991', '0000-0002-5413-9038'),
 ('0000-0002-6371-2991', '0000-0003-2571-9102'),
 ('0000-0002-6371-2991', '0000-0002-9084-552X'),
 ('0000-0002-6371-2991', '0000-0002-0948-1176'),
 ('0000-0002-6371-2991', '0000-0002-8431-081X'),
 ('0000-0002-1211-4238', '0000-0002-2888-4202'),
 ('0000-0002-1211-4238', '0000-0002-1030-0512'),
 ('0000-0002-1211-4238', '0000-0002-5413-9038'),
 ('0000-0002-1211-42

In [127]:
rsd.build_network_pyvis("/Users/jennifer/RCSNA/data/rsd.db", "project")

'html file is created'

In [128]:
rsd.build_network_pyvis("/Users/jennifer/RCSNA/data/rsd.db", "software")

'html file is created'

In [160]:
rsd.build_network_gephi("/Users/jennifer/RCSNA/data/rsd.db", "project")


'gexf file is created'

In [161]:
rsd.build_network_gephi("/Users/jennifer/RCSNA/data/rsd.db", "software")

'gexf file is created'

## Neo4j

In [15]:
from py2neo import Graph
from neo4j import GraphDatabase
import requests
from itertools import combinations

In [5]:
uri = "neo4j+s://a93e1a64.databases.neo4j.io"
user = "neo4j"
password = "ifVwf-BGTZt1ltrNgVbh5Q6aGSxQSksQJSxBUbJb1BE"


In [13]:
response_project = requests.get(
    "https://research-software-directory.org/api/v1/project?select=id,title,description"
    )
response_author = requests.get(
    "https://research-software-directory.org/api/v1/team_member?select=id,project,orcid,given_names,family_names, affiliation"
    )
response_software = requests.get(
    "https://research-software-directory.org/api/v1/software?select=id,concept_doi,brand_name, description"
    )
response_contributor = requests.get(
    "https://research-software-directory.org/api/v1/contributor?select=id,software,orcid,given_names,family_names,affiliation"
    )
    
projects = response_project.json()
authors_proj = response_author.json()
software = response_software.json()
contributor_soft = response_contributor.json()

for a in authors_proj:
    a['name'] = a['given_names'] + ' ' + a['family_names']
    if a['orcid'] == None:
        a['orcid'] = orcid.name_to_orcid_id(a['name'])

In [6]:
RSD = GraphDatabase.driver(uri, auth=(user, password))

In [35]:
authors_proj[0]

{'id': 'db1d71e8-4a3b-457a-b5d6-325ae5bce9be',
 'project': '2a94df7c-deb2-4278-b975-506f2c8a64f8',
 'orcid': '0000-0002-2662-1994',
 'given_names': 'Dafne',
 'family_names': 'van Kuppevelt',
 'affiliation': 'Netherlands eScience Center',
 'name': 'Dafne van Kuppevelt'}

In [27]:
def create_project_person(tx, author):
    if author['affiliation'] is None:
        author['affiliation'] = ''
    tx.run("MERGE (p:Person {orcid: $person.orcid, name: $person.name, affiliation: $person.affiliation, worktype: 'project'})",
            person = author)

In [22]:
def get_project_coauthor(rsd_db_path):
    # "/Users/jennifer/RCSNA/data/rsd.db"
    con = sqlite3.connect(rsd_db_path)
    cur = con.cursor()
    res = cur.execute("""
        SELECT GROUP_CONCAT(orcid, ',') AS orcids 
        FROM author 
        GROUP BY project 
        HAVING COUNT(*) > 1
        """)
    coauthor = res.fetchall()

    coauthor_links = []
    for au in coauthor:
        au = au[0].strip('(\')')
        au = au.split(',')
        coauthor_links = coauthor_links+list(combinations(au, 2))
    cur.close()

    weights = []
    for i in coauthor_links:
        weights.append(coauthor_links.count(i))

    return coauthor_links, weights

In [18]:
coauthor_links, weights = get_project_coauthor("/Users/jennifer/RCSNA/data/rsd.db")

In [32]:


# MATCH (a:Person)
# WITH a
# MATCH (b:Person) 
# WHERE a.orcid = $link_a AND b.orcid = $link_b
# MERGE (a)-[r:COAUTHOR {worktype: 'project', weight: $W}]-(b)

def add_project_edge(tx, link, weight):
    tx.run("""
        MATCH 
            (a:Person), 
            (b:Person)
        WHERE a.orcid = $link_a AND b.orcid = $link_b
        MERGE (a)-[r:COAUTHOR {worktype: 'project', weight: $W}]-(b)
        """, 
        link_a=link[0], link_b=link[1], W = weight
    )
    

In [34]:
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    with driver.session(database="neo4j") as session:
        # Create some nodes
        for author in authors_proj:
            session.execute_write(create_project_person, author)

        # Create some relationships
        for i in range(len(coauthor_links)):
            session.execute_write(add_project_edge, coauthor_links[i], weights[i])

In [29]:
len(coauthor_links)

4502

In [30]:
from collections import Counter
len(Counter(coauthor_links))

3998

In [ ]:
session.close()
driver.close()

#### Get orcid for RSD authors

In [219]:
import requests
import pandas as pd
import sys
sys.path.append("..")
from rcn_py import neo4j_rsd


In [2]:
# name = "Maria Chertova"
# affiliation = "Netherlands eScience Center"
name = "John Doe"
affiliation = "University of California, Los Angeles"


In [19]:
ror_id = "00rbjv475"        # no record
grid_id = "grid.454309.f"       # 33 results
crossref_funder_id = "100013407"        # no record
affiliation_org_name = "Netherlands eScience Center"        # much more data than I thought
ringgold_org_id = ""


In [92]:
headers = {
    "Accept": "application/vnd.orcid+json",
}

# query_url = f"https://pub.orcid.org/v3.0/search/?q=name:{name}+AND+affiliation-org-name:{affiliation}"


# params = (("q", "name:{name}),)
response = requests.get(
    f"https://pub.orcid.org/v3.0/search/?q=grid-org-id:{grid_id}", headers=headers
    # f"https://pub.orcid.org/v3.0/search/?q=affiliation-org-name:{affiliation_org_name}", headers=headers
    # f"https://pub.orcid.org/v3.0/search/?q=fundref-org-id:{crossref_funder_id}", headers=headers
)
temp = response.json()
    # if len(temp):
    #     orcid_id = temp["result"][0]["orcid-identifier"]["path"]
   

In [93]:
temp


{'result': [{'orcid-identifier': {'uri': 'https://orcid.org/0000-0001-8792-6305',
    'path': '0000-0001-8792-6305',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-6465-5589',
    'path': '0000-0002-6465-5589',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-8431-081X',
    'path': '0000-0002-8431-081X',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-2717-9851',
    'path': '0000-0002-2717-9851',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-2932-3028',
    'path': '0000-0002-2932-3028',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-7635-9533',
    'path': '0000-0002-7635-9533',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/0000-0003-2225-1428',
    'path': '0000-0003-2225-1428',
    'host': 'orcid.org'}},
  {'orcid-identifier': {'uri': 'https://orcid.org/00

In [76]:
orcid = '0000-0002-5025-7862'

In [77]:
# get the works of one person
response2 = requests.get(
    f"https://pub.orcid.org/v3.0/{orcid}/works", headers=headers
)
works_data = response2.json()

In [78]:
works_data

{'last-modified-date': {'value': 1654040356185},
 'group': [{'last-modified-date': {'value': 1654040356185},
   'external-ids': {'external-id': [{'external-id-type': 'doi',
      'external-id-value': '10.5194/egusphere-egu21-9387',
      'external-id-normalized': {'value': '10.5194/egusphere-egu21-9387',
       'transient': True},
      'external-id-normalized-error': None,
      'external-id-url': {'value': 'https://doi.org/10.5194/egusphere-egu21-9387'},
      'external-id-relationship': 'self'}]},
   'work-summary': [{'put-code': 89903286,
     'created-date': {'value': 1614838570891},
     'last-modified-date': {'value': 1654040356185},
     'source': {'source-orcid': None,
      'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
       'path': '0000-0001-9884-1913',
       'host': 'orcid.org'},
      'source-name': {'value': 'Crossref'},
      'assertion-origin-orcid': None,
      'assertion-origin-client-id': None,
      'assertion-origin-name': None},
  

In [86]:
work_url = f"https://pub.orcid.org/v3.0/0000-0002-5025-7862/work/89903286"
response = requests.get(work_url, headers=headers)
work_data = response.json()
work_data

{'created-date': {'value': 1614838570891},
 'last-modified-date': {'value': 1654040356185},
 'source': {'source-orcid': None,
  'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
   'path': '0000-0001-9884-1913',
   'host': 'orcid.org'},
  'source-name': {'value': 'Crossref'},
  'assertion-origin-orcid': None,
  'assertion-origin-client-id': None,
  'assertion-origin-name': None},
 'put-code': 89903286,
 'path': '/0000-0002-5025-7862/work/89903286',
 'title': {'title': {'value': 'A multi-model ensemble weighting method (ClimWIP) in ESMValTool'},
  'subtitle': None,
  'translated-title': None},
 'journal-title': None,
 'short-description': None,
 'citation': None,
 'type': 'other',
 'publication-date': {'year': {'value': '2021'},
  'month': {'value': '03'},
  'day': {'value': '04'}},
 'external-ids': {'external-id': [{'external-id-type': 'doi',
    'external-id-value': '10.5194/egusphere-egu21-9387',
    'external-id-normalized': {'value': '10.5194/egusphere-egu

In [80]:

contributors = work_data["contributors"]['contributor']
for contributor in contributors:
        name = contributor["credit-name"]["value"]
        print(name)
        # work_data = response.json()
        # contributors = work_data["contributors"]
        # print(contributors)

        #     print(f"Contributors for work {work_data['title']['title']['value']}:")
        #     for contributor in contributors:
        #         name = contributor["credit-name"]
        #         print(f"- {name}")
        # else:
        #     print(f"Failed to retrieve contributors for work. Error message: {response.content}")

Ruth Lorenz
Lukas Brunner
Peter Kalverla
Stef Smeets
Jaro Camphuijsen
Bouwe Andela


In [42]:
response3 = requests.get(
    f"https://pub.orcid.org/v3.0/{orcid}", headers=headers
)
info = response3.json()

In [43]:
info

{'orcid-identifier': {'uri': 'https://orcid.org/0000-0001-8792-6305',
  'path': '0000-0001-8792-6305',
  'host': 'orcid.org'},
 'preferences': {'locale': 'en'},
 'history': {'creation-method': 'MEMBER_REFERRED',
  'completion-date': None,
  'submission-date': {'value': 1520522741331},
  'last-modified-date': {'value': 1678970261014},
  'claimed': True,
  'source': None,
  'deactivation-date': None,
  'verified-email': True,
  'verified-primary-email': True},
 'person': {'last-modified-date': None,
  'name': {'created-date': {'value': 1520522741331},
   'last-modified-date': {'value': 1520522741564},
   'given-names': {'value': 'Stijn'},
   'family-name': {'value': 'Heldens'},
   'credit-name': None,
   'source': None,
   'visibility': 'public',
   'path': '0000-0001-8792-6305'},
  'other-names': {'last-modified-date': None,
   'other-name': [],
   'path': '/0000-0001-8792-6305/other-names'},
  'biography': None,
  'researcher-urls': {'last-modified-date': None,
   'researcher-url': [],

In [245]:
neo4j_rsd.get_scopus_info_from_orcid('0000-0002-0655-6160')

('', '', '')

In [9]:
# Assume that we already have orcid ids, can we get their Scopus id?
for record in temp['result']:
    temp_orcid = record['orcid-identifier']['path']
    result = neo4j_rsd.get_scopus_info_from_orcid(temp_orcid)
    print(result)
# haha no

('', '', '')
('', '', '')
('', '', '')
('', '', '')
('24480411800', 'Rena Bakhshi', 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=24480411800&origin=inward')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('56493993600', 'Robin A. Richardson', 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=56493993600&origin=inward')
('', '', '')
('', '', '')
('', '', '')
('57204369541', 'Carla Lopes', 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=57204369541&origin=inward')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('', '', '')
('57097799500', 'Florian Huber', 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=57097799500&origin=inward')
('57214165039', 'Abel Soares Siqueira', 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=57214165039&origin=inward')
('', '', '')
('7006591871', 'Jason Maassen', 'https://www.scop

#### Get info from Scopus using DOI

In [81]:
MYAPIKEY = "3d120b6ddb7d069272dfc2bc68af4028"
doi = "10.5194%2Fegusphere-egu21-9387"

In [82]:
url = "http://api.elsevier.com/content/search/scopus?query=DOI%28%22"+doi+"%22%29"

header = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
resp = requests.get(url, headers=header)
doi_results = resp.json()

In [83]:
doi_results

{'search-results': {'opensearch:totalResults': '0',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '0',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'DOI("10.5194/egusphere-egu21-9387")',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=DOI%28%2210.5194%2Fegusphere-egu21-9387%22%29',
    '@type': 'application/json'}],
  'entry': [{'@_fa': 'true', 'error': 'Result set was empty'}]}}

#### Can we collect all the colleague info from Scopus directly?

Only 25 return values

In [97]:
AF_ID = "60159380"
url = "http://api.elsevier.com/content/search/author?query=AF-ID%28"+AF_ID+"%29"

header = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
resp = requests.get(url, headers=header)
scopus_aff_results = resp.json()

In [98]:
scopus_aff_results['search-results']['entry']

[{'@_fa': 'true',
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/author/author_id/57216997004'},
   {'@_fa': 'true',
    '@ref': 'search',
    '@href': 'https://api.elsevier.com/content/search/author?query=au-id%2857216997004%29'},
   {'@_fa': 'true',
    '@ref': 'scopus-citedby',
    '@href': 'https://www.scopus.com/author/citedby.uri?partnerID=HzOxMe3b&citedAuthorId=57216997004&origin=inward'},
   {'@_fa': 'true',
    '@ref': 'scopus-author',
    '@href': 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=57216997004&origin=inward'}],
  'prism:url': 'https://api.elsevier.com/content/author/author_id/57216997004',
  'dc:identifier': 'AUTHOR_ID:57216997004',
  'eid': '9-s2.0-57216997004',
  'preferred-name': {'surname': 'Diblen',
   'given-name': 'F.',
   'initials': 'F.'},
  'name-variant': [{'@_fa': 'true',
    'surname': 'Diblen',
    'given-name': 'Faruk',
    'initials': 'F.'},
   {'@_fa': 'true',
    'surname': 'D

#### Can we get all the info from Scopus using `name` and `aff`?

Yes

In [605]:
neo4j_rsd.get_scopus_info_from_orcid('0000-0002-0655-6160')

('', '', '')

In [793]:
author_firstname = "Djura"
last = "Ahmed"
AF_ID = "60159380"
url = "http://api.elsevier.com/content/search/author?query=AUTHFIRST%28+"+author_firstname+"%29+AND+AF-ID%28"+AF_ID+"%29"
#+AND+AF-ID%28"+AF_ID+"%29
# url = "http://api.elsevier.com/content/search/author?query=AUTHFIRST%28+"+author_firstname+"%29+AND+AUTHLASTNAME%28"+last+"%29"
header = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
resp = requests.get(url, headers=header)
scopus_author_search = resp.json()

In [12]:
from rcn_py import orcid as orcid_py

In [13]:
myorcid = orcid_py.name_to_orcid_id("Barbara M. I. Vreede")

In [14]:
myorcid

'0000-0002-5023-4601'

In [4]:
orcid = "0000-0002-5025-7862"
# url = "https://api.openalex.org/authors/https://orcid.org/"+orcid
url = "https://api.openalex.org/works?filter=author.id:A2313063375"
header = {'Accept' : 'application/json'}
resp = requests.get(url, headers=header)
openalex_search = resp.json()
openalex_search

{'meta': {'count': 45, 'db_response_time_ms': 33, 'page': 1, 'per_page': 25},
 'results': [{'id': 'https://openalex.org/W2909524579',
   'doi': 'https://doi.org/10.5194/wes-4-193-2019',
   'title': 'Low-level jets over the North Sea based on ERA5 and observations: together they do better',
   'display_name': 'Low-level jets over the North Sea based on ERA5 and observations: together they do better',
   'publication_year': 2019,
   'publication_date': '2019-04-03',
   'ids': {'openalex': 'https://openalex.org/W2909524579',
    'doi': 'https://doi.org/10.5194/wes-4-193-2019',
    'mag': '2909524579'},
   'language': 'en',
   'primary_location': {'is_oa': True,
    'landing_page_url': 'https://doi.org/10.5194/wes-4-193-2019',
    'pdf_url': None,
    'source': {'id': 'https://openalex.org/S4210189655',
     'display_name': 'Wind energy science',
     'issn_l': '2366-7443',
     'issn': ['2366-7451', '2366-7443'],
     'host_organization': 'https://openalex.org/P4310313756',
     'host_org

In [64]:
aff = "University%20of%20Amsterdam"
url = "https://api.openalex.org/institutions?search="+aff
header = {'Accept' : 'application/json'}
resp = requests.get(url, headers=header)
openalex_aff_search = resp.json()
openalex_aff_search

{'meta': {'count': 3, 'db_response_time_ms': 10, 'page': 1, 'per_page': 25},
 'results': [{'id': 'https://openalex.org/I887064364',
   'ror': 'https://ror.org/04dkp9463',
   'display_name': 'University of Amsterdam',
   'relevance_score': 130258.9,
   'country_code': 'NL',
   'type': 'education',
   'homepage_url': 'http://www.uva.nl/en/home',
   'image_url': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Uva%C2%AEmerken%20ENG.png',
   'image_thumbnail_url': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Uva%C2%AEmerken%20ENG.png&width=300',
   'display_name_acronyms': ['UvA'],
   'display_name_alternatives': [],
   'repositories': [{'id': 'https://openalex.org/S4306400088',
     'display_name': 'UvA-DARE (University of Amsterdam)',
     'host_organization': 'https://openalex.org/I887064364',
     'host_organization_name': 'University of Amsterdam',
     'host_organization_lineage': ['https://openalex.org/I887064364']}],
   'works_count':

In [31]:
openalex_aff_search['results'][0]

{'id': 'https://openalex.org/I4210095242',
 'ror': 'https://ror.org/00rbjv475',
 'display_name': 'Netherlands eScience Center',
 'relevance_score': 11740.813,
 'country_code': 'NL',
 'type': 'nonprofit',
 'homepage_url': 'https://www.esciencecenter.nl/',
 'image_url': None,
 'image_thumbnail_url': None,
 'display_name_acronyms': ['NLeSC'],
 'display_name_alternatives': [],
 'repositories': [],
 'works_count': 475,
 'cited_by_count': 15192,
 'summary_stats': {'2yr_mean_citedness': 4.08256880733945,
  'h_index': 43,
  'i10_index': 150},
 'ids': {'openalex': 'https://openalex.org/I4210095242',
  'ror': 'https://ror.org/00rbjv475',
  'grid': 'grid.454309.f'},
 'geo': {'city': 'Amsterdam',
  'geonames_city_id': '2759794',
  'region': None,
  'country_code': 'NL',
  'country': 'Netherlands',
  'latitude': 52.35695,
  'longitude': 4.954451},
 'international': {'display_name': {'en': 'Netherlands eScience Center'}},
 'associated_institutions': [{'id': 'https://openalex.org/I2800991832',
   'ro

In [63]:
url = 'https://api.openalex.org/works?filter=institutions.id:I887064364'
# institution_id = url.split('institutions.id:')[-1]
header = {'Accept' : 'application/json'}

params = {
        'per_page': 25,  # Number of results per page
        'page': 1       # Initial page number
    }

resp = requests.get(url, headers=header, params=params)
work_search_per_page = resp.json()
work_search_per_page

# openalex_aff_search = []
# while True:
#         resp = requests.get(url, headers=header, params=params)
#         work_search_per_page = resp.json()
#         openalex_aff_search.extend(work_search_per_page['results'])
#         total_results = work_search_per_page['meta']['count']
#         current_page = work_search_per_page['meta']['page']
#         results_per_page = work_search_per_page['meta']['per_page']

#         total_pages = (total_results + results_per_page - 1) // results_per_page
#         if current_page == total_pages:
#             break  # Break the loop when all pages have been retrieved

#         params['page'] += 1 

# openalex_aff_search

{'meta': {'count': 135366,
  'db_response_time_ms': 45,
  'page': 1,
  'per_page': 25},
 'results': [{'id': 'https://openalex.org/W3118615836',
   'doi': 'https://doi.org/10.1136/bmj.n71',
   'title': 'The PRISMA 2020 statement: an updated guideline for reporting systematic reviews',
   'display_name': 'The PRISMA 2020 statement: an updated guideline for reporting systematic reviews',
   'publication_year': 2021,
   'publication_date': '2021-03-29',
   'ids': {'openalex': 'https://openalex.org/W3118615836',
    'doi': 'https://doi.org/10.1136/bmj.n71',
    'mag': '3118615836',
    'pmid': 'https://pubmed.ncbi.nlm.nih.gov/33782057',
    'pmcid': 'https://www.ncbi.nlm.nih.gov/pmc/articles/8005924'},
   'language': 'en',
   'primary_location': {'is_oa': True,
    'landing_page_url': 'https://doi.org/10.1136/bmj.n71',
    'pdf_url': 'https://www.bmj.com/content/bmj/372/bmj.n71.full.pdf',
    'source': {'id': 'https://openalex.org/S192814187',
     'display_name': 'BMJ',
     'issn_l': '095

In [32]:
escience_work_search

{'meta': {'count': 491, 'db_response_time_ms': 42, 'page': 1, 'per_page': 25},
 'results': [{'id': 'https://openalex.org/W2302501749',
   'doi': 'https://doi.org/10.1038/sdata.2016.18',
   'title': 'The FAIR Guiding Principles for scientific data management and stewardship',
   'display_name': 'The FAIR Guiding Principles for scientific data management and stewardship',
   'publication_year': 2016,
   'publication_date': '2016-03-15',
   'ids': {'openalex': 'https://openalex.org/W2302501749',
    'doi': 'https://doi.org/10.1038/sdata.2016.18',
    'mag': '2302501749',
    'pmid': 'https://pubmed.ncbi.nlm.nih.gov/26978244',
    'pmcid': 'https://www.ncbi.nlm.nih.gov/pmc/articles/4792175'},
   'language': 'en',
   'primary_location': {'is_oa': True,
    'landing_page_url': 'https://doi.org/10.1038/sdata.2016.18',
    'pdf_url': None,
    'source': {'id': 'https://openalex.org/S2607323502',
     'display_name': 'Scientific Data',
     'issn_l': '2052-4463',
     'issn': ['2052-4463'],
   

1


In [22]:
doi = 'https://doi.org/10.5194/wes-4-193-2019'
author_list = []
orcid_string = []
concept_score = {}
concept_score['concept_name'] = []
concept_score['score'] = []
openalex_pub_result = openalex.get_pub_info_by_doi(doi)
title = openalex_pub_result['results'][0]["title"]
for au in openalex_pub_result['results'][0]["authorships"]:
    author_list.append(au['author']['display_name'])
    orcid_string.append(au['author']['orcid'])
author_string = ', '.join(author_list)
date = openalex_pub_result['results'][0]['publication_date']
source = openalex_pub_result['results'][0]['primary_location']["source"]
source_page = source["display_name"]+', '+ source['issn_l']
cited_by_count = openalex_pub_result['results'][0]["cited_by_count"]
concepts = openalex_pub_result['results'][0]['concepts']
for c in concepts[0:7]:
    concept_score['concept_name'].append(c['display_name'])
    concept_score['score'].append(c['score'])
cite_by_year = openalex_pub_result['results'][0]['counts_by_year']
print(author_string)
print(orcid_string)
print(date)
print(source_page)
print(title)
print(cited_by_count)
print(concept_score)
print(cite_by_year)


Peter Kalverla, James S. Duncan, Gert-Jan Steeneveld, Albert A. M. Holtslag
['https://orcid.org/0000-0002-5025-7862', None, 'https://orcid.org/0000-0002-5922-8179', None]
2019-04-03
Wind energy science, 2366-7443
Low-level jets over the North Sea based on ERA5 and observations: together they do better
39
{'concept_name': ['Wind speed', 'Classification of discontinuities', 'Data assimilation', 'Environmental science', 'Jet (fluid)', 'Diurnal cycle', 'Climatology'], 'score': [0.6310531, 0.58202386, 0.49342766, 0.49033427, 0.4753356, 0.44513568, 0.42882544]}
[{'year': 2023, 'cited_by_count': 3}, {'year': 2022, 'cited_by_count': 15}, {'year': 2021, 'cited_by_count': 12}, {'year': 2020, 'cited_by_count': 8}, {'year': 2019, 'cited_by_count': 1}]


In [926]:
openalex_search['works_api_url']

'https://api.openalex.org/works?filter=author.id:A2313063375'

In [920]:
for i in openalex_search["results"]:
    print(i["display_name"])
    print(i["doi"])

Low-level jets over the North Sea based on ERA5 and observations: together they do better
https://doi.org/10.5194/wes-4-193-2019
An observational climatology of anomalous wind events at offshore meteomast IJmuiden (North Sea)
https://doi.org/10.1016/j.jweia.2017.03.008
Evaluation of the Weather Research and Forecasting Model in the Durance Valley Complex Terrain during the KASCADE Field Campaign
https://doi.org/10.1175/jamc-d-15-0258.1
Quality of wind characteristics in recent wind atlases over the North Sea
https://doi.org/10.1002/qj.3748
A maximum entropy approach to the parametrization of subgrid processes in two‐dimensional flow
https://doi.org/10.1002/qj.2817
Evaluation of three mainstream numerical weather prediction models with observations from meteorological mast IJmuiden at the North Sea
https://doi.org/10.1002/we.2267
Clustering wind profile shapes to estimate airborne wind energy production
https://doi.org/10.5194/wes-5-1097-2020
The eWaterCycle platform for open and FAIR h

In [900]:
concept_score = {}
concept_score['concept_name'] = []
concept_score['score'] = []
for c in openalex_search['x_concepts'][0:6]:
    concept_score['concept_name'].append(c['display_name'])
    concept_score['score'].append(c['score'])
 

In [903]:
concept_score['concept_name']

['Physics',
 'Geography',
 'Biology',
 'Computer science',
 'Meteorology',
 'Engineering']

In [24]:
doi ="https://doi.org/10.5281/zenodo.5645154"
if "https://doi.org/" in doi:
    doi = doi.replace("https://doi.org/",'')
doi

'10.5281/zenodo.5645154'

In [200]:
for record in scopus_author_search['search-results']['entry']:
    scopus_id = record['dc:identifier'].split(':')[-1]
    name = record['preferred-name']['given-name'] + ' ' + record['preferred-name']['surname']
    print(name, scopus_id)

Peter Kalverla 57189467443


In [165]:
scopus_id = scopus_author_search['search-results']['entry'][0]['dc:identifier'].split(':')[-1]

'55940841200'

#### Let's have a try to combine data from these three sources

1. RSD: name, orcid(maybe empty), project(title), software(title, doi)
2. Crossref (few data): name, orcid, projects
3. Scopus (few data): name, orcid(maybe empty), scopus_id, publications

1. RSD 

In [134]:
response_project = requests.get(
        "https://research-software-directory.org/api/v1/project?select=id,title,subtitle,created_at,description"
        )
response_author = requests.get(
        "https://research-software-directory.org/api/v1/team_member?select=id,project,orcid,given_names,family_names, affiliation"
        )
response_software = requests.get(
        "https://research-software-directory.org/api/v1/software?select=id,concept_doi,brand_name,created_at,description"
        )
response_contributor = requests.get(
        "https://research-software-directory.org/api/v1/contributor?select=id,software,orcid,given_names,family_names,affiliation"
        )
        
projects = response_project.json()
authors_proj = response_author.json()
software = response_software.json()
contributor_soft = response_contributor.json()



In [166]:
rsd_author_names = []
rsd_authors = []
proj_id_list = []
au_proj_links = []
for au in authors_proj:
    if au["affiliation"] == "Netherlands eScience Center":
        name = au["given_names"] + " " + au["family_names"]
        orcid = au["orcid"]
        # Save the author 
        if name not in rsd_author_names:
            rsd_author_names.append(name)
            author_info = {}
            author_info["name"] = name
            author_info["first_name"] = au["given_names"]
            author_info["surname"] = au["family_names"]
            author_info["orcid"] = orcid
            rsd_authors.append(author_info)
        else:
            if orcid != None:
                for record in rsd_authors:
                    if record["name"] == name and record["orcid"] == None:
                        record["orcid"] = orcid
        # Save the project
        if au["project"] not in proj_id_list:
            proj_id_list.append(au["project"])
        
        au_proj_links.append({"author": name, "project": au["project"]})



In [167]:
soft_id_list = []
au_soft_links = []
for au in contributor_soft:
    if au["affiliation"] == "Netherlands eScience Center":
        name = au["given_names"] + " " + au["family_names"]
        orcid = au["orcid"]
        if name not in rsd_author_names:
            rsd_author_names.append(name)
            author_info = {}
            author_info["name"] = name
            author_info["first_name"] = au["given_names"]
            author_info["surname"] = au["family_names"]
            author_info["orcid"] = orcid
            rsd_authors.append(author_info)
        else:
            if orcid != None:
                for record in rsd_authors:
                    if record["name"] == name and record["orcid"] == None:
                        record["orcid"] = orcid
        # Save the software
        if au["software"] not in soft_id_list:
            soft_id_list.append(au["software"])

        au_soft_links.append({"author": name, "software": au["software"]})

In [168]:
rsd_authors
# 138

[{'name': 'Djura Smits',
  'first_name': 'Djura',
  'surname': 'Smits',
  'orcid': '0000-0003-4096-0260'},
 {'name': 'Niels  Drost',
  'first_name': 'Niels ',
  'surname': 'Drost',
  'orcid': '0000-0001-9795-7981'},
 {'name': 'Tom Klaver',
  'first_name': 'Tom',
  'surname': 'Klaver',
  'orcid': '0000-0001-9411-2107'},
 {'name': 'Rena Bakhshi',
  'first_name': 'Rena',
  'surname': 'Bakhshi',
  'orcid': '0000-0002-2932-3028'},
 {'name': 'Pushpanjali Pawar',
  'first_name': 'Pushpanjali',
  'surname': 'Pawar',
  'orcid': '0000-0002-0655-6160'},
 {'name': 'Maria Chertova',
  'first_name': 'Maria',
  'surname': 'Chertova',
  'orcid': None},
 {'name': 'Jisk Attema',
  'first_name': 'Jisk',
  'surname': 'Attema',
  'orcid': '0000-0002-0948-1176'},
 {'name': 'Dafne van Kuppevelt',
  'first_name': 'Dafne',
  'surname': 'van Kuppevelt',
  'orcid': '0000-0002-2662-1994'},
 {'name': 'Merijn Verstraaten',
  'first_name': 'Merijn',
  'surname': 'Verstraaten',
  'orcid': '0000-0002-2717-9851'},
 {'n

In [689]:
au_proj_links

[{'author': 'Djura Smits', 'project': '0c4cdce5-6ee4-4a21-8189-3f9119190b9d'},
 {'author': 'Niels  Drost', 'project': '8ceb016f-314b-462e-9757-e3dffdcef734'},
 {'author': 'Tom Klaver', 'project': '9f2aff31-b420-4aa1-a044-94b9980a282f'},
 {'author': 'Rena Bakhshi', 'project': '5a44afd4-d9aa-4d43-893d-6c7631fab63d'},
 {'author': 'Pushpanjali Pawar',
  'project': 'eb40a234-de21-4952-b19d-cc02d98a9a16'},
 {'author': 'Maria Chertova',
  'project': '80343f6c-ab3f-4c51-bc53-5b954fb2dd8f'},
 {'author': 'Jisk Attema', 'project': 'de1c3e58-a361-4a95-8a0e-5ceef2141db2'},
 {'author': 'Dafne van Kuppevelt',
  'project': '2a94df7c-deb2-4278-b975-506f2c8a64f8'},
 {'author': 'Merijn Verstraaten',
  'project': '80343f6c-ab3f-4c51-bc53-5b954fb2dd8f'},
 {'author': 'Stef Smeets', 'project': '37ffe404-52dd-4895-8cd3-eda9e67447e3'},
 {'author': 'Jason Maassen',
  'project': 'b76ea788-f394-461f-a1f2-d24a131bfbc9'},
 {'author': 'Sonja Georgievska',
  'project': '29635ebd-a24e-4e76-bdec-88f8b7ebc56d'},
 {'auth

In [685]:
au_soft_links

[{'author': 'Ben van Werkhoven',
  'software': '25ac7630-b959-4446-a484-84ebab43232b'},
 {'author': 'Ben van Werkhoven',
  'software': '153f4ae9-a031-4525-a257-908dbddca5ef'},
 {'author': 'Giulia Crocioni',
  'software': 'c51a6038-a08f-4483-a308-7441f0d7d4ff'},
 {'author': 'Mateusz Kuzak',
  'software': '135132b5-54ea-4264-a58e-c0d73e8dbdf7'},
 {'author': 'Alessio Sclocco',
  'software': '8b8d8c43-0931-45f7-8e77-a32bb796d6e1'},
 {'author': 'Oscar Martinez Rubi',
  'software': '0945ccca-53af-4312-8ef8-5120491110cf'},
 {'author': 'Tom Klaver', 'software': 'ddfd3168-40a7-4b34-a287-16910a007b24'},
 {'author': 'Johan Hidding',
  'software': 'ed7278c4-812a-466f-ab84-84bc91d7ef1e'},
 {'author': 'Francesco Nattino',
  'software': '9b8ec850-eb99-45a0-a39d-852099b5a8ff'},
 {'author': 'Jaro Camphuijsen',
  'software': '153f4ae9-a031-4525-a257-908dbddca5ef'},
 {'author': 'Lars Buitinck',
  'software': 'e5a4509d-8486-4c0c-a53b-48f71a16bd93'},
 {'author': 'Pranav Chandramouli',
  'software': 'c51a60

In [220]:
df = pd.DataFrame(rsd_authors) 
# saving the dataframe 
df.to_csv('esc_people.csv') 

In [106]:
# make it be a dictionary
rsd_author_dict = {}
for au in rsd_authors:
    rsd_author_dict[au["name"]] = au["orcid"]

In [169]:
rsd_author_dict



{'Djura Smits': '0000-0003-4096-0260',
 'Niels  Drost': '0000-0001-9795-7981',
 'Tom Klaver': '0000-0001-9411-2107',
 'Rena Bakhshi': '0000-0002-2932-3028',
 'Pushpanjali Pawar': '0000-0002-0655-6160',
 'Maria Chertova': None,
 'Jisk Attema': '0000-0002-0948-1176',
 'Dafne van Kuppevelt': '0000-0002-2662-1994',
 'Merijn Verstraaten': '0000-0002-2717-9851',
 'Stef Smeets': '0000-0002-5413-9038',
 'Jason Maassen': '0000-0002-8172-4865',
 'Sonja Georgievska': '0000-0002-8094-4532',
 'Patrick Bos': '0000-0002-6033-960X',
 'Yifat Dzigan': '0000-0002-0935-0088',
 'Romulo Gonçalves': '0000-0003-2225-1428',
 'Vedran Kasalica': '0000-0002-0097-1056',
 'Piter de Boer': None,
 'Jens Wehner': None,
 'Sven van der Burg': '0000-0003-1250-6968',
 'Souley Madougou': '0000-0002-8493-5860',
 'Pablo Lopez-Tarifa': '0000-0002-4136-1860',
 'Adriënne Mendrik': '0000-0001-6631-7068',
 'Meiert Grootes': '0000-0002-5733-4795',
 'Pranav Chandramouli': '0000-0002-7896-2969',
 'Dani Bodor': '0000-0003-2109-2349',

#### Let's try to get all the eScience info by name and AF-ID

In [212]:
import time

In [224]:
def store_scopus_id_by_name_aff(record):
        # author_name = record['name']
        author_firstname = record['first_name']
        author_surname = record['surname']

        AF_ID = "60159380" # eScience id

        url = "http://api.elsevier.com/content/search/author?query=AUTHFIRST%28+"+author_firstname+"%29+AND+AF-ID%28"+AF_ID+"%29"
        #+AND+AF-ID%28"+AF_ID+"%29
        header = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
        resp = requests.get(url, headers=header)
        scopus_author_search = resp.json()
        if 'error' not in scopus_author_search['search-results']['entry'][0].keys():
            
            for record in scopus_author_search['search-results']['entry']:

                if record['preferred-name']['surname'] == author_surname:

                    scopus_id = record['dc:identifier'].split(':')[-1]

                    # name = record['preferred-name']['given-name'] + ' ' + record['preferred-name']['surname']

                    # print(name, scopus_id)
                    record['scopus_id'] = scopus_id
                    
        return record

In [226]:


for record in rsd_authors:
    time.sleep(1)

    if record["orcid"] != None:
        # use the orcid function built before directly 
        get_scopus_id = neo4j_rsd.get_scopus_info_from_orcid(record["orcid"])
        if  get_scopus_id != ('', '', ''):
            record['scopus_id'] = get_scopus_id[2]
        else:
            store_scopus_id_by_name_aff(record)

    else:
        # get info by first name and aff
        store_scopus_id_by_name_aff(record)
        # author_name = record['name']
        # author_firstname = record['first_name']
        # author_surname = record['surname']

        # AF_ID = "60159380" # eScience id

        # url = "http://api.elsevier.com/content/search/author?query=AUTHFIRST%28+"+author_firstname+"%29+AND+AF-ID%28"+AF_ID+"%29"
        # #+AND+AF-ID%28"+AF_ID+"%29
        # header = {'Accept' : 'application/json', 
        #                 'X-ELS-APIKey' : MYAPIKEY}
        # resp = requests.get(url, headers=header)
        # scopus_author_search = resp.json()
        # if 'error' not in scopus_author_search['search-results']['entry'][0].keys():
            
        #     for record in scopus_author_search['search-results']['entry']:
        #         if record['preferred-name']['surname'] == author_surname:
        #             scopus_id = record['dc:identifier'].split(':')[-1]

        #             name = record['preferred-name']['given-name'] + ' ' + record['preferred-name']['surname']

        #             print(name, scopus_id)

In [227]:
rsd_authors

[{'name': 'Djura Smits',
  'first_name': 'Djura',
  'surname': 'Smits',
  'orcid': '0000-0003-4096-0260'},
 {'name': 'Niels  Drost',
  'first_name': 'Niels ',
  'surname': 'Drost',
  'orcid': '0000-0001-9795-7981'},
 {'name': 'Tom Klaver',
  'first_name': 'Tom',
  'surname': 'Klaver',
  'orcid': '0000-0001-9411-2107'},
 {'name': 'Rena Bakhshi',
  'first_name': 'Rena',
  'surname': 'Bakhshi',
  'orcid': '0000-0002-2932-3028',
  'scopus_id': ('24480411800',
   'Rena Bakhshi',
   'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=24480411800&origin=inward')},
 {'name': 'Pushpanjali Pawar',
  'first_name': 'Pushpanjali',
  'surname': 'Pawar',
  'orcid': '0000-0002-0655-6160'},
 {'name': 'Maria Chertova',
  'first_name': 'Maria',
  'surname': 'Chertova',
  'orcid': None},
 {'name': 'Jisk Attema',
  'first_name': 'Jisk',
  'surname': 'Attema',
  'orcid': '0000-0002-0948-1176',
  'scopus_id': ('8071168700',
   'J. J. Attema',
   'https://www.scopus.com/authid/detail.uri?par

In [ ]:
# Save projects and software properties
rsd_projects = []
author_proj_link = []
for proj in projects:
    if proj["id"] in proj_id_list:
        title = proj["title"]
        created_at = proj["created_at"]
        description = proj["description"]
        rsd_projects.append({"rsd_id": proj["id"],
                            "title": proj["title"],
                            "subtitle": proj["subtitle"],
                            "created_at": proj["created_at"],
                            "description": proj["description"]})
        


In [683]:
esc_author = pd.read_csv("esc_people.csv", dtype=object, index_col=0)
esc_author

,name,first_name,surname,orcid,scopus_id
0,Djura Smits,Djura,Smits,0000-0003-4096-0260,57339115500
1,Niels Drost,Niels,Drost,0000-0001-9795-7981,15065185400
2,Tom Klaver,Tom,Klaver,0000-0001-9411-2107,57103436500
3,Rena Bakhshi,Rena,Bakhshi,0000-0002-2932-3028,24480411800
4,Pushpanjali Pawar,Pushpanjali,Pawar,0000-0002-0655-6160,NaN
...,...,...,...,...,...
127,Adam Belloum,Adam,Belloum,0000-0001-6306-6937,6603093115
128,Niek de Jonge,Niek,de Jonge,0000-0002-3054-6210,57803280200
129,Romulo Goncalves,Romulo,Goncalves,0000-0003-2225-1428,34880075200
130,Ewan Jacov Cahen,Ewan Jacov,Cahen,NaN,57193025740


In [742]:
import numpy as np
if not pd.isnull(esc_author['orcid'][5]):
    print(1)
else:
    print(2)
    

2


In [741]:
esc_author['orcid'][5]

nan

First of all, save all person node

In [745]:
def save_esc_author(tx, author):
    if not pd.isnull(author['scopus_id']):
        if not pd.isnull(author['orcid']):
            tx.run("""
                MERGE (p:Person {scopus_id: $scopus_id})
                SET p.orcid= $orcid, 
                    p.affiliation = $aff,
                    p.name= $name
                """,
                scopus_id = author['scopus_id'], 
                orcid = author['orcid'],
                aff = "Netherlands eScience Center",
                name = author['name']
            )
        else:
            tx.run("""
                MERGE (p:Person {scopus_id: $scopus_id})
                SET 
                    p.affiliation = $aff,
                    p.name= $name
                """,
                scopus_id = author['scopus_id'], 
                aff = "Netherlands eScience Center",
                name = author['name']
            )
    else:
        if not pd.isnull(author['orcid']):
            tx.run("""
                MERGE (p:Person {orcid: $orcid})
                SET p.orcid= $orcid, 
                    p.affiliation = $aff,
                    p.name= $name
                """,
                orcid = author['orcid'],
                aff = "Netherlands eScience Center",
                name = author['name']
            )
        else:
            tx.run("""
                MERGE (p:Person {name: $name})
                SET 
                    p.affiliation = $aff,
                    p.name= $name
                """,
                aff = "Netherlands eScience Center",
                name = author['name']
            )

In [748]:
for pl in au_proj_links:
        project_id = pl['project']
        for p in projects:
            if project_id == p['id']:
                  tx.run("""
                        MERGE (p:Project {project_id: $project_id})
                        SET 
                        p.title = $title,
                        p.subtitle= $subtitle.
                        p.year = $year,
                        p.description = $description
                        """,
                        aff = "Netherlands eScience Center",
                        name = author['name']
                )

In [752]:
import re

In [767]:
def save_rsd_pub_nodes(tx, authors, projects, proj_link):
    # project
    for pl in proj_link:
        project_id = pl['project']
        au_name = re.sub('\(.*?\)','',pl['author'])
        au_name = au_name.replace('  ', ' ')
        for p in projects:
            if project_id == p['id']:
                tx.run("""
                        MERGE (p:Project {project_id: $project_id})
                        SET 
                        p.title = $title,
                        p.subtitle= $subtitle,
                        p.year = $year,
                        p.description = $description
                        """,
                        project_id = project_id,
                        title = p['title'],
                        subtitle = p['subtitle'],
                        year = p['created_at'][0:4],
                        description = p['description']
                    )
                
        for index,author in authors.iterrows():
            if author['name'] == au_name:
                if not pd.isnull(author['scopus_id']):
                    tx.run("""
                        MATCH 
                            (n:Person {scopus_id: $scopus_id}),
                            (s:Project {project_id: $project_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(s)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = s.title,
                            r.year = s.year
                        """,
                        scopus_id = author['scopus_id'],
                        project_id = project_id,
                        author_name = au_name
                    )
                elif not pd.isnull(author['orcid']):
                    tx.run("""
                        MATCH 
                            (n:Person {orcid: $orcid}),
                            (s:Project {project_id: $project_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(s)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = s.title,
                            r.year = s.year
                        """,
                        orcid = author['orcid'],
                        project_id = project_id,
                        author_name = au_name
                    )
                else:
                    tx.run("""
                        MATCH 
                            (n:Person {name: $author_name}),
                            (s:Project {project_id: $project_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(s)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = s.title,
                            r.year = s.year
                        """,
                        project_id = project_id,
                        author_name = au_name
                    )
    # Save all coauthors of the project
            

In [775]:
a=0
for s in software:
    if s['brand_name'] is None:
        a = a+1
a

0

In [778]:
def save_rsd_soft_nodes(tx, authors, software, soft_link):
    # project
    for sl in soft_link:
        software_id = sl['software']
        au_name = re.sub('\(.*?\)','',sl['author'])
        au_name = au_name.replace('  ', ' ')
        for s in software:
            if software_id == s['id']:
                if s['description'] is None:
                    s['description'] = ''
                if not s['concept_doi'] is None:
                    tx.run("""
                            MERGE (p:Software {software_id: $software_id})
                            SET 
                            p.doi = $doi,
                            p.title= $title,
                            p.year = $year,
                            p.description = $description
                            """,
                            software_id = software_id,
                            doi = s['concept_doi'],
                            title = s['brand_name'],
                            year = s['created_at'][0:4],
                            description = s['description']
                        )
                else:
                    tx.run("""
                            MERGE (p:Software {software_id: $software_id})
                            SET 
                            p.title= $title,
                            p.year = $year,
                            p.description = $description
                            """,
                            software_id = software_id,
                            title = s['brand_name'],
                            year = s['created_at'][0:4],
                            description = s['description']
                        )
                
        for index,author in authors.iterrows():
            if author['name'] == au_name:
                if not pd.isnull(author['scopus_id']):
                    tx.run("""
                        MATCH 
                            (n:Person {scopus_id: $scopus_id}),
                            (s:Software {software_id: $software_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(s)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = s.title,
                            r.year = s.year
                        """,
                        scopus_id = author['scopus_id'],
                        software_id = software_id,
                        author_name = au_name
                    )
                elif not pd.isnull(author['orcid']):
                    tx.run("""
                        MATCH 
                            (n:Person {orcid: $orcid}),
                            (s:Software {software_id: $software_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(s)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = s.title,
                            r.year = s.year
                        """,
                        orcid = author['orcid'],
                        software_id = software_id,
                        author_name = au_name
                    )
                else:
                    tx.run("""
                        MATCH 
                            (n:Person {name: $author_name}),
                            (s:Software {software_id: $software_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(s)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = s.title,
                            r.year = s.year
                        """,
                        software_id = software_id,
                        author_name = au_name
                    )

Find pubs from scopus for everyone

In [822]:
scopus_id = "57339115500"
url = "http://api.elsevier.com/content/search/author?query=AU-ID%28+"+scopus_id+"%29"
        #+AND+AF-ID%28"+AF_ID+"%29
header = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
resp = requests.get(url, headers=header)
scopus_author_search = resp.json()

In [823]:
scopus_author_search


{'search-results': {'opensearch:totalResults': '1',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '1',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'AU-ID( 57339115500)',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/author?start=0&count=25&query=AU-ID%28+57339115500%29',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/author?start=0&count=25&query=AU-ID%28+57339115500%29',
    '@type': 'application/json'}],
  'entry': [{'@_fa': 'true',
    'link': [{'@_fa': 'true',
      '@ref': 'self',
      '@href': 'https://api.elsevier.com/content/author/author_id/57339115500'},
     {'@_fa': 'true',
      '@ref': 'search',
      '@href': 'https://api.elsevier.com/content/search/author?query=au-id%2857339115500%29'},
     {'@_fa': 'true',
      '@ref': 'scopus-citedby',
      '@href': 'https://www.scopus.com/

In [825]:
doi = '10.3233/SHTI220682'
url_doc = f"http://api.elsevier.com/content/search/scopus?query=DOI%28+"+doi+"%29"
header_doc = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
resp_doc = requests.get(url_doc, headers=header_doc)
scopus_doc_info = resp_doc.json()
scopus_doc_info

{'search-results': {'opensearch:totalResults': '1',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '1',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'DOI( 10.3233/SHTI220682)',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=DOI%28+10.3233%2FSHTI220682%29',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=DOI%28+10.3233%2FSHTI220682%29',
    '@type': 'application/json'}],
  'entry': [{'@_fa': 'true',
    'link': [{'@_fa': 'true',
      '@ref': 'self',
      '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85133231172'},
     {'@_fa': 'true',
      '@ref': 'author-affiliation',
      '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85133231172?field=author,affiliation'},
     {'@_fa': 'true',
      '@ref': 'scopus',
 

In [859]:
def save_scopus_pub(tx, author):
    if not pd.isnull(author['scopus_id']):
        scopus_id = author["scopus_id"]

        # The following tx def is not necessary, just reduce the computation because it usually occur a bug after 1-hour running
        def check(tx, scopus_id):
            return list(tx.run("""
                MATCH (n:Person {scopus_id: $scopus_id})-[r]->(p:Publication)<-[r2]-(m:Person) 
                WHERE m.affiliation <> "Netherlands eScience Center" 
                RETURN p
                """,
                scopus_id = scopus_id
                ))
        with GraphDatabase.driver(uri, auth=(user, password)) as driver:
            driver.verify_connectivity()
            with driver.session(database="rsd") as session:
                results = session.execute_read(check, scopus_id)
                if results:
                    return

        # ###### Find all pubs of this author #########
        url = "http://api.elsevier.com/content/search/scopus?query=AU-ID%28+"+scopus_id+"%29"
        header = {'Accept' : 'application/json', 
                'X-ELS-APIKey' : MYAPIKEY}
        resp = requests.get(url, headers=header)
        scopus_author_info = resp.json()
        for doc in scopus_author_info["search-results"]["entry"]:
            title = doc['dc:title']
            if 'prism:doi' not in doc.keys():
                continue
            doi = doc['prism:doi']
            cite_count = doc['citedby-count']
            year = doc['prism:coverDate'][0:4]

            # ######## write into the dataset
            tx.run("""
                MERGE (p:Publication {doi: $doi})
                SET p.title = $title,
                    p.year = $year,
                    p.citation_count = $citation_count
                """,
                doi = doi,
                title = title,
                year = year,
                citation_count = cite_count
                )
            
            tx.run("""
                MATCH 
                    (n:Person {scopus_id: $scopus_id}),
                    (p:Publication {doi: $doi})
                MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                ON CREATE SET 
                    r.author_name = $author_name,
                    r.title = p.title,
                    r.year = p.year
                """,
                scopus_id = scopus_id,
                doi = doi,
                author_name = author["name"]
            )
            
            # ######### Now we can get authors info using DOI we got #########
            url_doc = f"http://api.elsevier.com/content/search/scopus?query=DOI%28+"+doi+"%29&field=author&count=1"
            header_doc = {'Accept' : 'application/json', 
                            'X-ELS-APIKey' : MYAPIKEY}
            resp_doc = requests.get(url_doc, headers=header_doc)
            scopus_doc_info = resp_doc.json()
            scopus_doc_info
            if "author" not in scopus_doc_info["search-results"]["entry"][0].keys():
                continue
            for co_author in scopus_doc_info["search-results"]["entry"][0]["author"]:
                co_scopus_id = co_author['authid']
                if co_scopus_id == scopus_id:
                    continue
                if co_author["given-name"]:
                    name = co_author["given-name"] + ' ' + co_author["surname"]
                elif co_author["initials"]:
                    name = co_author["surname"]+ ' ' + co_author["initials"]
                else:
                    name = co_author["surname"]
                
                url = "http://api.elsevier.com/content/search/author?query=AU-ID%28+"+co_scopus_id+"%29"
                header = {'Accept' : 'application/json', 
                                'X-ELS-APIKey' : MYAPIKEY}
                resp = requests.get(url, headers=header)
                scopus_author_search = resp.json()
                try:
                    aff = scopus_author_search["search-results"]["entry"][0]["affiliation-current"]["affiliation-name"]
                    country = scopus_author_search["search-results"]["entry"][0]["affiliation-current"]["affiliation-country"]

                    tx.run("""
                        MERGE (p:Person {scopus_id: $scopus_id})
                        SET p.scopus_id= $scopus_id, 
                            p.affiliation = $aff,
                            p.name= $name,
                            p.country = $country
                        """,
                        scopus_id = co_scopus_id,
                        aff = aff,
                        name = name,
                        country = country
                    )
                except KeyError:
                    tx.run("""
                        MERGE (p:Person {scopus_id: $scopus_id})
                        SET p.scopus_id= $scopus_id, 
                            p.name= $name
                        """,
                        scopus_id = co_scopus_id,
                        name = name
                    )
                    
                tx.run("""
                    MATCH 
                        (n:Person {scopus_id: $scopus_id}),
                        (p:Publication {doi: $doi})
                    MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                    ON CREATE SET 
                        r.author_name = $author_name,
                        r.title = p.title,
                        r.year = p.year
                    """,
                    scopus_id = co_scopus_id,
                    doi = doi,
                    author_name = name
                )




Get all coauthors of projects and software in rsd

In [863]:
def save_rsd_coauthor(tx, author_project, contributor_soft, au_proj_links, au_soft_links):
    for pl in au_proj_links:
        proj_id = pl["project"]
        for au_proj in author_project:
            if proj_id == au_proj["project"]:
                orcid = au_proj["orcid"]
                affiliation = au_proj["affiliation"]
                name = au_proj["given_names"]+" "+au_proj["family_names"]
                name = re.sub('\(.*?\)','',name)
                name = name.replace('  ', ' ')
                
                if not pd.isnull(orcid):
                    tx.run("""
                        MERGE (p:Person {orcid: $orcid})
                        SET p.orcid= $orcid, 
                            p.affiliation = $aff,
                            p.name= $name
                        """,
                        orcid = orcid,
                        aff = affiliation,
                        name = name
                    )

                    tx.run("""
                        MATCH 
                            (n:Person {orcid: $orcid}),
                            (p:Project {project_id: $project_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = p.title,
                            r.year = p.year
                        """,
                        orcid = orcid,
                        project_id = proj_id,
                        author_name = name
                    )
                else:
                    tx.run("""
                        MERGE (p:Person {name: $name})
                        SET  
                            p.affiliation = $aff,
                            p.name= $name
                        """,
                        aff = affiliation,
                        name = name
                    )

                    tx.run("""
                        MATCH 
                            (n:Person {name: $author_name}),
                            (p:Project {project_id: $project_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = p.title,
                            r.year = p.year
                        """,
                        project_id = proj_id,
                        author_name = name
                    )
                

    for sl in au_soft_links:
        soft_id = sl["software"]
        for au_soft in contributor_soft:
            if soft_id == au_soft["software"]:
                orcid = au_soft["orcid"]
                affiliation = au_soft["affiliation"]
                name = au_soft["given_names"]+" "+au_soft["family_names"]
                name = re.sub('\(.*?\)','',name)
                name = name.replace('  ', ' ')
                
                if not pd.isnull(orcid):
                    tx.run("""
                        MERGE (p:Person {orcid: $orcid})
                        SET p.orcid= $orcid, 
                            p.affiliation = $aff,
                            p.name= $name
                        """,
                        orcid = orcid,
                        aff = affiliation,
                        name = name
                    )
                    tx.run("""
                        MATCH 
                            (n:Person {orcid: $orcid}),
                            (p:Software {software_id: $software_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                        ON CREATE SET 
                            r.author_name = $author_name,
                            r.title = p.title,
                            r.year = p.year
                        """,
                        orcid = orcid,
                        software_id = soft_id,
                        author_name = name
                    )
                else:
                    tx.run("""
                        MERGE (p:Person {name: $name})
                        SET  
                            p.affiliation = $aff,
                            p.name= $name
                        """,
                        aff = affiliation,
                        name = name
                    )
                    tx.run("""
                        MATCH 
                            (n:Person {name: $name}),
                            (p:Software {software_id: $software_id})
                        MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                        ON CREATE SET 
                            r.author_name = $name,
                            r.title = p.title,
                            r.year = p.year
                        """,
                        software_id = soft_id,
                        name = name
                    )

            
                

In [698]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
user = "neo4j"
password = "zhiningbai"

In [699]:
check_verify =  GraphDatabase.driver(uri, auth=(user, password))
check_verify.verify_connectivity()

In [864]:
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    with driver.session(database="rsd") as session:
        # for index,author in esc_author.iterrows():
        #     session.execute_write(save_esc_author, author)
        # session.execute_write(save_rsd_soft_nodes, esc_author, software, au_soft_links)
            # session.execute_write(save_scopus_pub, author)

        session.execute_write(save_rsd_coauthor, authors_proj, contributor_soft, au_proj_links, au_soft_links)

In [ ]:
def add_constraint(tx):
#     tx.run("""
#             CREATE CONSTRAINT doi IF NOT EXISTS
#             FOR (p:Publication) REQUIRE p.doi IS UNIQUE
#             """)
    tx.run("""
            CREATE CONSTRAINT scopus_id IF NOT EXISTS
            FOR (n:Person) REQUIRE n.scopus_id IS UNIQUE
            """)
    tx.run("""
            CREATE CONSTRAINT orcid IF NOT EXISTS
            FOR (n:Person) REQUIRE n.orcid IS UNIQUE
            """)